In [2]:
import numpy as np
import os
import pandas as pd
import plotly
import plotly.graph_objs as go
import plotly.offline as py
import plotly.express as px
from scipy import stats

# Indicadores

In [3]:
# Importa a biblioteca do sqlite, cria a conexão e o ponteiro
import sqlite3
con = sqlite3.connect("database\\indicadores.db")
cur = con.cursor()

In [4]:
# Pegando os dados relevantes presentes na database
df_ = pd.read_sql("SELECT * FROM regions_df", con)
df2_ = pd.read_sql("SELECT * FROM brazil_df", con)
df3_ = pd.read_sql("SELECT * FROM states_df", con)

In [5]:
states = list(df3_.state.unique())
df3_.head()

,index,state,ibge_code,week_last_day,epidemiological_week,total_inhabitants,last_available_confirmed,last_available_deaths,new_week_cases,new_week_deaths,prevalence,mortality,lethality,incidence_cases,incidence_deaths,cases_growth_factor,deaths_growth_factor
0,104301,RO,11.0,2020-03-21,12,1.796460e+06,3,0,3,0,0.166995,0.000000,0.000000,0.166995,0.000000,0.000000,0.0
1,104304,RO,11.0,2020-03-28,13,1.796460e+06,6,0,3,0,0.333990,0.000000,0.000000,0.166995,0.000000,1.000000,0.0
2,104308,RO,11.0,2020-04-04,14,1.796460e+06,11,1,5,1,0.612315,0.055665,9.090909,0.278325,0.055665,1.666667,0.0
3,104316,RO,11.0,2020-04-11,15,1.796460e+06,34,2,23,1,1.892611,0.111330,5.882353,1.280296,0.055665,4.600000,1.0
4,104326,RO,11.0,2020-04-18,16,1.796460e+06,128,4,94,2,7.125124,0.222660,3.125000,5.232513,0.111330,4.086957,2.0


In [6]:
# Obter os dados relevantes de cada macro_região e os armazená-los em uma lista de dataframes
def get_states(state, relevant_dataframe):
    output_df = {}
    
    for i in state:
        output_df[i] = relevant_dataframe[(relevant_dataframe['state']==i)]
        output_df[i].set_index('week_last_day', inplace=True)
    
    return output_df

In [7]:
# Encontrar o valor máximo de uma coluna numa lista de dataframes
def find_max(lista_de_dataframes,column):
    max_value = 0
    for i in range(0,len(lista_de_dataframes)):
        aux = lista_de_dataframes[i][column].max()
        if(aux>max_value):
            max_value = aux
    return max_value

In [8]:
last_available_confirmed_st = df3_[['state','week_last_day','last_available_confirmed']].copy()
casos_acumulados = get_states(states,last_available_confirmed_st)

In [9]:
casos_acumulados["RO"]

,state,last_available_confirmed
week_last_day,,
2020-03-21,RO,3
2020-03-28,RO,6
2020-04-04,RO,11
2020-04-11,RO,34
2020-04-18,RO,128
2020-04-25,RO,328
2020-05-02,RO,654
2020-05-09,RO,1263
2020-05-16,RO,1918


# Mobilidade

## Lendo a base de dados

In [74]:
df = pd.read_csv('base_csv/States_Mobility.csv')

Função para obter um dicionário que acessa a variação da mobilidade conforme um dia

In [11]:
def getData(estado:str, coluna:str):
    '''
        Input: Uma string correspondendo ao nome da capital de interesse e outra correspondendo a coluna
        de interesse.
        
        Output: Um dicionário que tem os dias como chave e a variação da mobilidade como valores.
    '''
    teste = df[df["iso_3166_2_code"] == estado]
    return dict(zip(list(teste['date']), list(teste[coluna])))

In [12]:
df["iso_3166_2_code"] = [i[3:] for i in df["iso_3166_2_code"]]

In [13]:
df

,Unnamed: 0,country_region_code,country_region,sub_region_1,sub_region_2,metro_area,iso_3166_2_code,census_fips_code,date,retail_and_recreation_percent_change_from_baseline,grocery_and_pharmacy_percent_change_from_baseline,parks_percent_change_from_baseline,transit_stations_percent_change_from_baseline,workplaces_percent_change_from_baseline,residential_percent_change_from_baseline,epidemiological_week
0,0,BR,Brazil,Federal District,NaN,NaN,DF,NaN,2020-02-23,-6.0,-2.0,-18.0,-6.0,-7.0,1.0,9
1,1,BR,Brazil,Federal District,NaN,NaN,DF,NaN,2020-02-24,-31.0,-15.0,-16.0,-34.0,-61.0,12.0,9
2,2,BR,Brazil,Federal District,NaN,NaN,DF,NaN,2020-02-25,-39.0,-27.0,14.0,-35.0,-71.0,11.0,9
3,3,BR,Brazil,Federal District,NaN,NaN,DF,NaN,2020-02-26,-20.0,-7.0,-14.0,-17.0,-22.0,9.0,9
4,4,BR,Brazil,Federal District,NaN,NaN,DF,NaN,2020-02-27,-4.0,5.0,16.0,5.0,17.0,-1.0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5665,5665,BR,Brazil,State of Tocantins,NaN,NaN,TO,NaN,2020-09-15,NaN,NaN,NaN,NaN,-3.0,9.0,38
5666,5666,BR,Brazil,State of Tocantins,NaN,NaN,TO,NaN,2020-09-16,NaN,NaN,NaN,NaN,-3.0,9.0,38
5667,5667,BR,Brazil,State of Tocantins,NaN,NaN,TO,NaN,2020-09-17,NaN,NaN,NaN,NaN,-4.0,9.0,38
5668,5668,BR,Brazil,State of Tocantins,NaN,NaN,TO,NaN,2020-09-18,NaN,NaN,NaN,NaN,0.0,9.0,38


### Configuração do Layout

In [14]:
layout = go.Layout(width=1200,
    height=350,
    margin=dict(l=200, r=100, t=60, b=20),
    #paper_bgcolor="black",
    #geo=dict(bgcolor= 'rgba(0,0,0,0)')
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    title_font=dict(size=20),
    legend=dict(orientation="v",yanchor="bottom",x = 0.05,y=0.80,xanchor="left"),
    
    
    xaxis=dict(
        showline=False,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        ticks='outside', 
        linewidth=2,
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        )
    ),
    
    yaxis=dict(
        showline=False,
        showgrid=False,
        showticklabels=True,
        linecolor='rgb(204, 204, 204)',
        ticks='outside', 
        linewidth=2,
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        )
    ),
    
    yaxis2=dict(
        showline=False,
        showgrid=False,
        showticklabels=True,
        side="right",
        linecolor='rgb(204, 204, 204)',
        ticks='inside',
        linewidth=2,
        tickfont=dict(
            family='Arial',
            size=12,
            color='rgb(82, 82, 82)',
        )
    )
                  
    )

### Listas com cidades e colunas que se deseja visualizar

Há um problema com Brasília, que por algum motivo não é encontrada em sub_region_2.

O problema não é de digitação.

In [15]:
# CRIADO NA SEÇÃO INDICADORES
states

['RO',
 'AC',
 'AM',
 'RR',
 'PA',
 'AP',
 'TO',
 'MA',
 'PI',
 'CE',
 'RN',
 'PB',
 'PE',
 'AL',
 'SE',
 'BA',
 'MG',
 'ES',
 'RJ',
 'SP',
 'PR',
 'SC',
 'RS',
 'MS',
 'MT',
 'GO',
 'DF']

Verificando se as cidades da lista acima estão no DataFrame

In [16]:
for state in states:
    print(state)
    print(state in [i[3:] for i in df["iso_3166_2_code"].unique()])

RO
False
AC
False
AM
False
RR
False
PA
False
AP
False
TO
False
MA
False
PI
False
CE
False
RN
False
PB
False
PE
False
AL
False
SE
False
BA
False
MG
False
ES
False
RJ
False
SP
False
PR
False
SC
False
RS
False
MS
False
MT
False
GO
False
DF
False


Listando colunas com as variações de mobilidade para cada tipo de ambiente

In [17]:
percentChangeColumns = [
    "retail_and_recreation_percent_change_from_baseline",
    "grocery_and_pharmacy_percent_change_from_baseline",
    "parks_percent_change_from_baseline",
    "transit_stations_percent_change_from_baseline",
    "workplaces_percent_change_from_baseline",
    "residential_percent_change_from_baseline"
]

In [18]:
indices = [
    "last_available_confirmed",
    "last_available_deaths",
    "new_week_cases",
    "new_week_deaths",
    "prevalence",
    "mortality",
    "lethality",
    "incidence_cases",
    "incidence_deaths",
    "cases_growth_factor",
    "deaths_growth_factor"
]

## Criando uma função para plotar o gráfico

Função que recebe uma coluna do gráfico e retorna um nome mais legível correspondente a ela, a fim de acoplar ao título do gráfico.

In [19]:
def columnName(coluna:str):
    if coluna == "retail_and_recreation_percent_change_from_baseline":
        return "Varejo e Lazer"
    if coluna == "grocery_and_pharmacy_percent_change_from_baseline":
        return "Mercados e Farmácias"
    if coluna == "parks_percent_change_from_baseline":
        return "Parques"
    if coluna == "transit_stations_percent_change_from_baseline":
        return "Estações de Transporte Público"
    if coluna == "workplaces_percent_change_from_baseline":
        return "Locais de Trabalho"
    if coluna == "residential_percent_change_from_baseline":
        return "Residencial"

In [20]:
def indicName(coluna:str):
    if coluna == "last_available_confirmed":
        return "Casos Acumulados"
    if coluna == "last_available_deaths":
        return "Óbitos Acumulados"
    if coluna == "new_week_cases":
        return "Novos Casos"
    if coluna == "new_week_deaths":
        return "Novos Óbitos"
    if coluna == "prevalence":
        return "Prevalência"
    if coluna == "mortality":
        return "Mortalidade"
    if coluna == "lethality":
        return "Letalidade"
    if coluna == "incidence_cases":
        return "Incidência de Casos"
    if coluna == "incidence_deaths":
        return "Incidência de Mortes"
    if coluna == "cases_growth_factor":
        return "Fator de Crescimento de Casos"
    if coluna == "deaths_growth_factor":
        return "Fator de Crescimento de Mortes"

Essa função recebe um nome de cidade e uma coluna do DataFrame e, a partir disso, projeta a variação no valor contido nessa coluna para cada dia, com relação à "baseline".

Informações sobre a caracterização geral dos dados estão disponíveis no README dessa pasta.

In [21]:
def plotCapital(capital:str, coluna:str, col_bar:str):
    coluna_indicador = df3_[['state','week_last_day',col_bar]].copy()
    casos = get_states(states,coluna_indicador)
    dados = getData(capital, coluna)
    fig = go.Figure(layout=layout)
    fig.add_trace(go.Bar(
            x = casos[capital].index,
            y = casos[capital][col_bar],
            name=indicName(col_bar),
            yaxis='y2',
            hoverinfo = "all",
            marker_color ='green',
            marker_line_width=1.5,
            marker_line_color='rgb(8,48,107)',
            opacity=0.5
        ))
    fig.add_trace(go.Scatter(
            mode='lines+markers',
            x = list(dados.keys()),
            y = list(dados.values()),
            hoverinfo = "all",
            hoveron = "points",
            yaxis = "y",
            fill = "tonexty",
            name='Mudança percentual em '+ columnName(coluna) + ' - ' + capital,
            marker_color='crimson'
        ))

  
    fig.update_yaxes(visible=True, showticklabels=True)
    fig.update_xaxes(title='', visible=True, showticklabels=True)
    
    #Título
    update = []
    titulo = 'Mudança percentual em '+ columnName(coluna) + ' - ' + capital
    tipografia = dict(family='Arial', size=25, color='rgb(37,37,37)')
    update.append(
        dict(xref='paper', yref='paper', x=0.15, y=1, xanchor='left', yanchor='bottom',text=titulo, font=tipografia,showarrow=False)
    )
    
    #"Rodape"
    fonte = "Covid Data Analytics"
    tipografia2 = dict(family='Arial', size=12, color='rgb(150,150,150)')
    update.append(
        dict(xref='paper', yref='paper', x=0.445, y=-0.20, xanchor='left', yanchor='bottom',text=fonte, font=tipografia2,showarrow=False)
    )

    if str(dados["2020-08-17"])=='nan':
        #"Rodape"
        fonte = "*Os dados de Mobilidade referentes a " + columnName(coluna) + " a partir de 17/08/2020 não estão disponíveis"
        tipografia2 = dict(family='Arial', size=12, color='rgb(150,150,150)')
        update.append(
            dict(xref='paper', yref='paper', x=0.445, y=-0.25, xanchor='left', yanchor='bottom',text=fonte, font=tipografia2,showarrow=False)
        )
    #Max value
    maximo = max(list(dados.values()))
    update.append(dict(xref='x', yref='y', x=list(dados.keys())[list(dados.values()).index(maximo)], y=maximo,
                                      xanchor='right', yanchor='auto',
                                      text='{:.2f}'.format(max(list(dados.values()))),
                                      font=dict(family='Impact bold',
                                                size=13,
                                                ),
                                      showarrow=True, arrowhead=1,
                                      arrowcolor = 'black',
                                      bordercolor="#A9A9A9",
                                      borderwidth=2,
                                      borderpad=4,
                                      bgcolor="#F0FFFF",
                                      opacity=0.95))

    #Min value
    minimo = min(list(dados.values()))
    update.append(dict(xref='x', yref='y', x=list(dados.keys())[list(dados.values()).index(minimo)], y=minimo,
                                      xanchor='right', yanchor='auto',
                                      text='{:.2f}'.format(min(list(dados.values()))),
                                      font=dict(family='Impact bold',
                                                size=13,
                                                ),
                                      showarrow=True, arrowhead=1,
                                      arrowcolor = 'black',
                                      bordercolor="#A9A9A9",
                                      borderwidth=2,
                                      borderpad=4,
                                      bgcolor="#F0FFFF",
                                      opacity=0.95))

    fig.update_layout(annotations=update)
    
    coluna = coluna.split("_percent")[0]

    py.plot(fig, filename=os.getcwd() + '/Views/' + capital + '/' + col_bar + '/' + capital + '_' + col_bar + '_'+ coluna + '.html', auto_open = False)

#### Criando diretórios para armazenar os gráficos

In [21]:
# define the name of the directory to be created
os.mkdir("Views")
for coluna in states:
    os.mkdir('Views/' + coluna)
    for indice in indices:
        os.mkdir('Views/' + coluna+'/'+indice)

## Produzindo os gráficos

In [137]:
plotCapital("AC","grocery_and_pharmacy_percent_change_from_baseline",'last_available_confirmed')

In [201]:
for state in states:
    for coluna in percentChangeColumns:
        for indice in indices:
            plotCapital(state, coluna,indice)

# CORRELAÇÃO

## Preparando dados dos indicadores e mobilidade

In [22]:
# dados dos indicadores
coluna_indicador_ = df3_.drop(["ibge_code", "index"], axis=1).copy()
# dados mobilidade
dados_ = df.drop(["Unnamed: 0","country_region_code","country_region","sub_region_1","sub_region_2", "metro_area","census_fips_code"],axis=1).copy()

## Função para plotar gráficos de correlação

In [72]:
def plotRelacao(estado:str,mobilidade:str,indicador:str, lag:int, plot:bool, show: bool):
    
    # seleciona os dados referentes ao estado escolhido
    coluna_indicador_state = coluna_indicador_[coluna_indicador_.state==estado].copy()
    dados_state = dados_[dados_["iso_3166_2_code"] == "SP"]
    
    # lista dos ultimos dias de cada semana epidemiologica
    week_last_day = list(coluna_indicador_state.week_last_day.values).copy()
    # gerar lista com o dia de cada semana com maior mobilidade
    dia_maior = []
    maior = -100
    mudou = False
    for i in dados_state.index:
        if dados_state[mobilidade][i] > maior:
            maior = dados_state[mobilidade][i]
            dia = dados_state.date[i]
            mudou = True
        if dados_state['date'][i] in week_last_day:
            if mudou == False:
                break
            dia_maior.append(dia)
            maior = -100
            mudou = False
    # unir os dados de mobilidade e indicador em um dataframe
    if lag!=0:
        df_state = dados_state.set_index('date').T[dia_maior].T.reset_index()[["date",mobilidade]][:-lag].copy()
    else:
        df_state = dados_state.set_index('date').T[dia_maior].T.reset_index()[["date",mobilidade]].copy()
    df_state[indicador] = coluna_indicador_state[indicador].values[lag:(len(dia_maior))]
    df_state["epidemiological_week"] = coluna_indicador_state['epidemiological_week'].values[lag:(len(dia_maior))]
    
    if plot==True or show==True:
        # plotar gráfico com x indicador e y local de referencia para mobilidade
        titulo = mobilidade + " x " + indicador +" em "+ estado
        fig = go.Figure()
        fig = px.scatter(df_state,x=indicador, y=mobilidade, hover_data=["epidemiological_week"], title = titulo )
        fig.update_traces(mode='markers')
        # gráfico com 2 linhas, uma para indicador e uma para mobilidade
        titulo2 = mobilidade + " x " + indicador +" em "+ estado
        fig2 = go.Figure(layout = layout)
        fig2.add_trace(go.Scatter(x=df_state["epidemiological_week"], y=df_state[indicador], name = indicador, yaxis = "y2"))# hover_data=["epidemiological_week"],
        fig2.add_trace(go.Scatter(x=df_state["epidemiological_week"], y=df_state[mobilidade], name = mobilidade, yaxis = "y"))# hover_data=["epidemiological_week"],
        fig2.update_traces(mode='lines+markers')
        update = []
        tipografia = dict(family='Arial', size=18, color='rgb(37,37,37)')
        update.append(
            dict( xref='paper', yref='paper', x=0, y=1, xanchor='left', yanchor='bottom',text=titulo2, font=tipografia,showarrow=False)
        )
        fig2.update_layout(annotations=update)   
        
        if plot == True:
            py.plot(fig,filename=os.getcwd() +'/Graphs_Corr/'+ estado+'/'+indicador+'/'+'graph'+"_"+mobilidade+"_"+indicador+"_"+estado+'.html', auto_open=False)
            py.plot(fig2,filename=os.getcwd() +'/Graphs_Corr/'+ estado+'/'+indicador+'/'+'graph'+"_lines"+"_"+mobilidade+"_"+indicador+"_"+estado+'.html', auto_open=False)
        if show==True:
            fig.show()
            fig2.show()
    # calcular correlação de pearson e retornar somente o coeficiente de correlação, a função retorna um par (coeficiente, p-value)
    corr = stats.pearsonr(df_state[indicador],df_state[mobilidade])
    return corr[0]

## Criar pasta para armazenar gráficos

In [198]:
import os

# define the name of the directory to be created
os.mkdir("Graphs_Corr")
for coluna in states:
    os.mkdir('Graphs_Corr/' + coluna)
    for indice in indices:
        os.mkdir('Graphs_Corr/' + coluna+'/'+indice)

FileExistsError: [WinError 183] Não é possível criar um arquivo já existente: 'Graphs_Corr'

## Plotar gráficos e criar matriz de correlação

In [67]:
def plotcorr(show:bool,plot:bool, lag:int):
    i=0
    j=0
    corr = np.zeros((6,11))
    for coluna in percentChangeColumns:
        for indice in indices:
            corr[i][j] = plotRelacao('SP',coluna, indice, lag, plot,show)
            j+=1
        i+=1
        j=0
    return corr

In [68]:
corr = plotcorr(False,False, 0)
corr1 = plotcorr(False,False, 1)
corr2 = plotcorr(False,False, 2)

In [73]:
plotRelacao("SP","grocery_and_pharmacy_percent_change_from_baseline",'last_available_confirmed',2,False,True)

0.46325638313636086

## Tornar a matriz em um dataframe

In [69]:
correlacao = pd.DataFrame(corr,index=percentChangeColumns, columns= indices)
correlacao1 = pd.DataFrame(corr1,index=percentChangeColumns, columns= indices)
correlacao2 = pd.DataFrame(corr2,index=percentChangeColumns, columns= indices)

correlacao

,last_available_confirmed,last_available_deaths,new_week_cases,new_week_deaths,prevalence,mortality,lethality,incidence_cases,incidence_deaths,cases_growth_factor,deaths_growth_factor
retail_and_recreation_percent_change_from_baseline,0.012652,-0.039852,-0.072666,-0.306439,0.012652,-0.039852,-0.914360,-0.072666,-0.306439,0.487443,-0.703580
grocery_and_pharmacy_percent_change_from_baseline,0.331041,0.345555,0.398061,0.271040,0.331041,0.345555,-0.398917,0.398061,0.271040,0.036716,-0.643666
parks_percent_change_from_baseline,-0.010234,-0.045553,-0.036997,-0.239456,-0.010234,-0.045553,-0.804448,-0.036997,-0.239456,0.244887,-0.678449
transit_stations_percent_change_from_baseline,0.057803,0.013728,-0.009291,-0.236372,0.057803,0.013728,-0.902745,-0.009291,-0.236372,0.477896,-0.726101
workplaces_percent_change_from_baseline,0.362492,0.366007,0.346928,0.167489,0.362492,0.366007,-0.806468,0.346928,0.167489,0.201540,-0.766615
residential_percent_change_from_baseline,-0.382425,-0.364108,-0.298609,-0.073979,-0.382425,-0.364108,0.841605,-0.298609,-0.073979,-0.279997,0.595343


In [70]:
correlacao1

,last_available_confirmed,last_available_deaths,new_week_cases,new_week_deaths,prevalence,mortality,lethality,incidence_cases,incidence_deaths,cases_growth_factor,deaths_growth_factor
retail_and_recreation_percent_change_from_baseline,0.027140,-0.046049,-0.111913,-0.379839,0.027140,-0.046049,-0.816664,-0.111913,-0.379839,0.768876,-0.238825
grocery_and_pharmacy_percent_change_from_baseline,0.486062,0.463749,0.426499,0.262185,0.486062,0.463749,-0.465604,0.426499,0.262185,0.039869,-0.267952
parks_percent_change_from_baseline,0.075492,0.009355,-0.048606,-0.299396,0.075492,0.009355,-0.799122,-0.048606,-0.299396,0.675345,-0.254911
transit_stations_percent_change_from_baseline,0.088487,0.020156,-0.039424,-0.314771,0.088487,0.020156,-0.820116,-0.039424,-0.314771,0.728738,-0.264649
workplaces_percent_change_from_baseline,0.357521,0.349087,0.279028,0.075737,0.357521,0.349087,-0.785603,0.279028,0.075737,0.415134,-0.318248
residential_percent_change_from_baseline,-0.345143,-0.315760,-0.214661,0.036548,-0.345143,-0.315760,0.775164,-0.214661,0.036548,-0.459444,0.417624


In [71]:
correlacao2

,last_available_confirmed,last_available_deaths,new_week_cases,new_week_deaths,prevalence,mortality,lethality,incidence_cases,incidence_deaths,cases_growth_factor,deaths_growth_factor
retail_and_recreation_percent_change_from_baseline,-0.026799,-0.111350,-0.186097,-0.477794,-0.026799,-0.111350,-0.654526,-0.186097,-0.477794,0.787285,0.182546
grocery_and_pharmacy_percent_change_from_baseline,0.463256,0.433258,0.334170,0.210541,0.463256,0.433258,-0.434877,0.334170,0.210541,0.150151,-0.014019
parks_percent_change_from_baseline,0.013902,-0.061477,-0.138416,-0.408839,0.013902,-0.061477,-0.767093,-0.138416,-0.408839,0.698939,0.000269
transit_stations_percent_change_from_baseline,0.045702,-0.036048,-0.116236,-0.406670,0.045702,-0.036048,-0.669725,-0.116236,-0.406670,0.754771,0.172475
workplaces_percent_change_from_baseline,0.353887,0.330082,0.252337,-0.020772,0.353887,0.330082,-0.754997,0.252337,-0.020772,0.480870,0.033649
residential_percent_change_from_baseline,-0.364800,-0.317879,-0.172169,0.112082,-0.364800,-0.317879,0.659668,-0.172169,0.112082,-0.480686,-0.150256
